<a href="https://colab.research.google.com/github/LucasPerroni/alura-leitor-pdf/blob/main/project_imersao_alura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Criar o modelo

Instalando o SDK do google

In [1]:
!pip install -q -U google-generativeai

In [2]:
import google.generativeai as genai
from google.colab import userdata

GOOGLE_API_KEY = userdata.get("api_key")
genai.configure(api_key=GOOGLE_API_KEY)

Configurações do modelo

In [3]:
  generation_config = {
      "candidate_count": 1,
      "temperature": 0.5
  }

Configurações de segurança

In [4]:
safety_settings = {
    "HARASSMENT": "BLOCK_NONE",
    "HATE": "BLOCK_NONE",
    "SEXUAL": "BLOCK_NONE",
    "DANGEROUS": "BLOCK_NONE"
}

Inicializando o modelo

In [5]:
model = genai.GenerativeModel(model_name="gemini-1.0-pro",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

Criando o primeiro input

In [6]:
response = model.generate_content("Diga olá para o mundo!")
print(response.text)

Olá, mundo!


Criar o Chat Bot

In [7]:
chat = model.start_chat(history=[])

In [ ]:
# prompt = input(f"Esperando o prompt: ")

# while prompt != "fim":
#   response = chat.send_message(prompt)
#   print(f"\033[1;34mResposta:\033[0m {response.text}\n\n")
#   prompt = input(f"Esperando o prompt: ")

Histórico do Chat

In [ ]:
# import textwrap
# from IPython.display import display, Markdown

# def to_markdown(text):
#   text = text.replace('•', '  *')
#   return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# #Imprimindo o histórico
# for message in chat.history:
#   display(to_markdown(f'**{message.role}**: {message.parts[0].text}'))
#   if message.role == "model":
#     print('------------------------------------------------------------------')

# Criar objeto do vestibular

In [8]:
!pip install -q -U PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.7 MB/s eta 0:00:00


Colocar o caminho para o arquivo pdf do vestibular

In [9]:
pdf_path = "./vestibular_2019.pdf"

Função para extrair as questões do vestibular

In [10]:
import PyPDF2  # Importa a biblioteca PyPDF2 para trabalhar com arquivos PDF.
import re  # Importa a biblioteca re para usar expressões regulares.

def extract_questions(pdf_path):  # Define uma função chamada extract_questions que recebe o caminho do arquivo PDF como argumento.
  """
  Esta função extrai questões de múltipla escolha de um arquivo PDF.
  """
  pdf_file = open(pdf_path, 'rb')  # Abre o arquivo PDF no modo de leitura binária.
  pdf_reader = PyPDF2.PdfReader(pdf_file)  # Cria um objeto PdfReader para ler o conteúdo do PDF.

  questions = []  # Inicializa uma lista vazia para armazenar as questões extraídas.
  current_question = None  # Inicializa uma variável para armazenar a questão atual que está sendo processada.

  skip_paragraph = False
  skip_lines = False

  for page_num in range(2, len(pdf_reader.pages)):  # Itera sobre as páginas do PDF, começando da página 2 (índice 1).
    page = pdf_reader.pages[page_num]  # Obtém o objeto da página atual.
    page_text = page.extract_text()  # Extrai o texto da página atual.

    lines = page_text.split('\n')  # Divide o texto da página em linhas.

    for line in lines:  # Itera sobre cada linha da página.
      match = re.match(r'^(\*?\*?)(\d+) - (.*)', line)  # Verifica se a linha corresponde ao padrão de uma pergunta (ex: "*1 - Qual a cor do céu?").

      if "O texto a seguir é referência para a" in line or "O excerto a seguir é referência para a" in line:
        skip_paragraph = True
        continue
      elif skip_paragraph and match:
        skip_paragraph = False

      if "INGLÊS" in line:
        skip_lines = False
        continue
      elif skip_lines or skip_paragraph:
        continue

      if match:  # Se a linha for uma pergunta:
        if current_question:  # Se já existe uma pergunta sendo processada, adiciona ela à lista de questões.
          questions.append(current_question)
        question_number = match.group(2)  # Extrai o número da pergunta.
        first_line = match.group(3).strip()  # Extrai o texto da primeira linha da pergunta.
        current_question = {"number": question_number, "text": first_line, "options": [], "correctOption": None}  # Cria um dicionário para armazenar as informações da pergunta.
      else:  # Se a linha não for uma pergunta:
        if current_question:  # Se existe uma pergunta sendo processada:
          match = re.match(r'^(\►)?([a-e])\)(.*)', line)  # Verifica se a linha corresponde ao padrão de uma opção (ex: "a) Azul").
          if match:  # Se a linha for uma opção:
            is_correct = match.group(1) == "►"  # Verifica se a opção é a correta (marcada com "►").
            option_letter = match.group(2)  # Extrai a letra da opção.
            option_text = match.group(3).strip()  # Extrai o texto da opção.

            # Verifica se a opção ocupa mais de uma linha:
            next_line_index = lines.index(line) + 1
            while next_line_index < len(lines) \
              and not (re.match(r'^(\►)?([a-e])\)(.*)', lines[next_line_index])\
              or re.match(r'^(\*?\*?)(\d+) - (.*)', lines[next_line_index])):
              option_text += " " + lines[next_line_index].strip()  # Adiciona o texto da próxima linha à opção.
              del lines[next_line_index]  # Remove a linha que foi adicionada à opção.

            if is_correct:  # Se a opção for a correta:
              current_question["correctOption"] = option_letter  # Armazena a letra da opção correta na pergunta.
            current_question["options"].append(option_text)  # Adiciona o texto da opção à lista de opções da pergunta.

            if (question_number == "82" and option_letter == "e") or (question_number == "90" and option_letter == "e"):
              skip_lines = True
          else:  # Se a linha não for uma opção:
            if line.strip():  # Se a linha não estiver vazia:
              current_question["text"] += " " + line.strip()  # Adiciona o texto da linha ao texto da pergunta.

  if current_question:  # Se ainda existe uma pergunta sendo processada, adiciona ela à lista de questões.
    questions.append(current_question)

  pdf_file.close()  # Fecha o arquivo PDF.
  return questions  # Retorna a lista de questões extraídas.


Criar o objeto com as questões e converter em JSON

In [11]:
import json

questions_data = extract_questions(pdf_path)
questions = json.dumps(questions_data)
questions

'[{"number": "01", "text": "No pre\\u00e2mbulo da Declara\\u00e7\\u00e3o Universal  dos Direitos Humanos, l\\u00ea -se: \\u201c[...] Considerando que o desconhecimento e o desprezo dos direitos do Homem conduziram a atos de barb\\u00e1rie que revoltam a consci\\u00eancia da Humanidade  e que o advento de um mundo em que os seres humanos sejam livres de falar e d e crer, libertos do terror e da mis\\u00e9ria, foi proclamado como a mais alta inspira\\u00e7\\u00e3o do Homem [...]\\u201d. (grifo nosso) A partir dos conhecimentos sobre o contexto hist\\u00f3rico dessa declara\\u00e7\\u00e3o, assinale a alternativa que indica os eventos hist\\u00f3ricos em que ocor reram \\u201catos de barb\\u00e1rie\\u201d no s\\u00e9culo XX, antes da publica\\u00e7\\u00e3o desse documento e que tiveram impacto na sua elabora\\u00e7\\u00e3o.", "options": ["Guerra Civil Russa e constru\\u00e7\\u00e3o da Cortina de Ferro na Europa.", "Primeira Guerra Mundial e limpeza \\u00e9tnica na Iugosl\\u00e1via.", "Guer

# Utilizar o modelo para estudar o objeto

Função para printar as mensagens melhor

In [12]:
import textwrap
from IPython.display import display, Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

Pedindo para o modelo explicar a primeira questão

In [15]:
response = chat.send_message(f"Você é um professor de cursinho preparatório para o vestibular e eu sou \
um dos seus alunos. Eu vou perguntar pontos sobre questões de vestibular e você vai me ajudar a entendê-los. \
Pontue todos os pontos importantes sobre a questão {questions_data[0]} e explique o motivo da resposta correta.")

for message in chat.history[-1:]:
  display(to_markdown(f'**{message.role}**: {message.parts[0].text}'))

> **model**: **Pontos importantes da questão:**
> 
> * O preâmbulo da Declaração Universal dos Direitos Humanos menciona "atos de barbárie" que ocorreram antes de sua publicação.
> * A questão pede para identificar os eventos históricos específicos que influenciaram a elaboração da declaração.
> 
> **Resposta correta: e) Segunda Guerra Mundial e Holocausto.**
> 
> **Justificativa:**
> 
> A Segunda Guerra Mundial e o Holocausto foram eventos terríveis que resultaram na morte de milhões de pessoas. Esses eventos demonstraram a necessidade urgente de proteger os direitos humanos e prevenir atrocidades semelhantes no futuro. A Declaração Universal dos Direitos Humanos foi elaborada em resposta a esses eventos e visa garantir que todos os seres humanos tenham direitos e liberdades fundamentais, independentemente de raça, religião, gênero ou qualquer outra distinção.
> 
> As outras opções estão incorretas porque:
> 
> * A Guerra Civil Russa e a construção da Cortina de Ferro ocorreram após a publicação da Declaração Universal dos Direitos Humanos.
> * A Primeira Guerra Mundial e a limpeza étnica na Iugoslávia não foram eventos tão significativos quanto a Segunda Guerra Mundial e o Holocausto em termos de impacto na elaboração da declaração.
> * A Guerra Civil Espanhola e os ataques terroristas da Al-Qaeda nos Estados Unidos ocorreram após a publicação da Declaração Universal dos Direitos Humanos.
> * A Guerra Russo-Japonesa e o genocídio dos tutsis em Ruanda também não foram eventos tão significativos quanto a Segunda Guerra Mundial e o Holocausto em termos de impacto na elaboração da declaração.

Pedir para ele explicar outra questão qualquer

In [16]:
response = chat.send_message(f"Agora me explique da mesma forma a questão {questions_data[64]}.")

for message in chat.history[-1:]:
  display(to_markdown(f'**{message.role}**: {message.parts[0].text}'))

> **model**: **Pontos importantes da questão:**
> 
> * A questão apresenta uma imagem com a divisão regional do Brasil e pede para identificar a alternativa correta sobre a orientação da rosa dos ventos.
> * A rosa dos ventos é uma convenção que indica os pontos cardeais (norte, sul, leste e oeste).
> 
> **Resposta correta: a) As indicações de norte “para cima” e sul “para baixo” são convenções e podem ser alteradas.**
> 
> **Justificativa:**
> 
> A orientação da rosa dos ventos é uma convenção e pode ser alterada dependendo do contexto. No caso do mapa do Brasil apresentado na questão, a rosa dos ventos está orientada com o norte para cima e o sul para baixo. No entanto, essa orientação poderia ser alterada sem afetar a precisão do mapa. Por exemplo, o mapa poderia ser girado 90 graus no sentido horário, de modo que o norte ficasse para a direita e o sul para a esquerda.
> 
> As outras opções estão incorretas porque:
> 
> * A Terra não obedece a um referencial magnético específico que determine que o norte da rosa dos ventos deva sempre apontar para a linha do Equador.
> * O mapa não precisa ser alterado para que as regiões sejam representadas corretamente. A orientação da rosa dos ventos pode ser alterada sem afetar a precisão do mapa.
> * O Trópico de Capricórnio não precisa ser substituído pelo Trópico de Câncer. A orientação da rosa dos ventos pode ser alterada sem afetar a precisão do mapa.
> * A rosa dos ventos pode ser alterada, mesmo que o mapa esteja invertido. A orientação da rosa dos ventos é uma convenção e pode ser alterada dependendo do contexto.

Limpar histórico

In [14]:
chat.history = []